In [4]:
from google.colab import userdata
secretkey = userdata.get('OPENAI')
# Set the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = secretkey
!pip install streamlit yfinance openai numpy
# install curl and wget
!apt-get install -y curl
#!apt-get install -y wget

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.21).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [6]:
!curl https://loca.lt/mytunnelpassword

35.229.75.152

In [11]:
# Set the OpenAI API key as an environment variable before running the Streamlit app
import os
from google.colab import userdata

openai_secret_key = userdata.get('OPENAI')
os.environ['OPENAI_API_KEY'] = openai_secret_key

!streamlit run app.py & npx localtunnel --port 8501

⠙

⠹⠸⠼⠴⠦
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.75.152:8501

⠧⠇⠏⠋⠙⠹⠸your url is: https://rare-cameras-build.loca.lt
  Stopping...
^C


In [10]:
%%writefile app.py
import streamlit as st
import yfinance as yf
import openai
import os

# --- Configuration ---
openai_api_key = os.environ.get('OPENAI_API_KEY')

if not openai_api_key:
    st.error("OpenAI API key not found. Please ensure the OPENAI_API_KEY environment variable is set.")
    st.stop()

client = openai.OpenAI(api_key=openai_api_key)


# --- Data Fetching ---
def fetch_stock_data(ticker, period):
    """Fetches stock history, recommendations, and news using yfinance."""
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period=period)
        recommendations = stock.recommendations
        news = stock.news
        info = stock.info

        price_change = 0
        if len(hist) > 1:
            price_change = ((hist['Close'].iloc[-1] - hist['Close'].iloc[-2]) / hist['Close'].iloc[-2]) * 100

        news_headlines = [item.get('title', 'No Title Available') for item in news[:5] if item.get('title')]
        news_text = ". ".join(news_headlines)

        return round(price_change, 2), hist, recommendations, news_text, info

    except Exception as e:
        st.error(f"Error fetching data for {ticker}: {e}")
        return None, None, None, None, None

# --- Mock Sentiment Score (Placeholder) ---
def mock_sentiment_score(ticker):
    """Generates a mock sentiment score. Replace with real sentiment analysis."""
    import numpy as np
    np.random.seed(hash(ticker) % (2**32))
    bullish = np.random.uniform(40, 80)
    bearish = 100 - bullish
    return round(bullish,1), round(bearish,1)

# --- Narrative Generation ---
def generate_narrative(ticker, price_change, bullish, bearish, news_text, recommendations_data):
    """Generates a market narrative using the OpenAI API."""
    try:
        persona_prompt = """
You are a senior financial market analyst and storyteller.
Explain stock price movements, insider trading, and social sentiment in a clear, concise, factual manner.
Avoid jargon, be unbiased, and highlight possible bubble risks or sustained trends.
"""
        few_shot_examples = """
Example 1:
Stock: AAPL
Price Change: +3.5%
Sentiment: 75% bullish
Recent News: 'Apple announces new chip with improved battery life'
Analyst Recommendations: Strong Buy: 5, Buy: 24, Hold: 14, Sell: 2, Strong Sell: 3
Narrative: Apple’s stock increased by 3.5% following the announcement of its new chip promising longer battery life. Market sentiment is strongly bullish, reflecting optimism on upcoming product demand. Analyst recommendations are predominantly positive (29 'Strong Buy' or 'Buy' vs. 5 'Sell' or 'Strong Sell'), reinforcing the positive outlook. However, valuation metrics suggest some caution for a short-term pullback.

Example 2:
Stock: TSLA
Price Change: -4.2%
Sentiment: 60% bearish
Recent News: 'Tesla faces regulatory scrutiny in Europe'
Analyst Recommendations: Strong Buy: 1, Buy: 5, Hold: 10, Sell: 8, Strong Sell: 4
Narrative: Tesla's 4.2% drop reflects growing concerns due to regulatory challenges in Europe as highlighted in recent news. Although sentiment tilts bearish, with more analysts recommending 'Hold' or 'Sell' (22 in total) compared to 'Strong Buy' or 'Buy' (6 in total), insider buying activity remains steady, which may moderate the downturn.
"""
        recommendation_summary = "Analyst Recommendations: "
        if recommendations_data is not None and not recommendations_data.empty:
            latest_rec = recommendations_data.iloc[0]
            recommendation_summary += f"Strong Buy: {latest_rec.get('strongBuy', 'N/A')}, Buy: {latest_rec.get('buy', 'N/A')}, Hold: {latest_rec.get('hold', 'N/A')}, Sell: {latest_rec.get('sell', 'N/A')}, Strong Sell: {latest_rec.get('strongSell', 'N/A')}"
        else:
            recommendation_summary += "No recent analyst recommendations available."


        prompt = f"""
{persona_prompt}

{few_shot_examples}

Stock: {ticker.upper()}
Price Change: {price_change}%
Sentiment: {bullish}% bullish, {bearish}% bearish
Recent News: '{news_text}'
{recommendation_summary}

Narrative:
"""
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": persona_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=500
        )
        return completion.choices[0].message.content.strip()

    except Exception as e:
        st.error(f"Error generating narrative: {e}")
        return "Could not generate narrative."

# --- General AI Chat Function ---
def general_ai_chat(prompt):
    """Generates a response to a general chat prompt using the OpenAI API."""
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini", # Using a potentially faster and cheaper model for chat
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=200 # Set a reasonable limit for chat responses
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        st.error(f"Error generating chat response: {e}")
        return "Could not generate a response."


# --- Streamlit UI ---
st.set_page_config(layout="wide", page_title="MarketWhisper: AI Financial Storyteller")

st.title("MarketWhisper: AI Financial Storyteller")
st.subheader("Get AI-driven narratives on stock movements, sentiment, and analyst views.")

# --- General AI Chat Section ---
st.subheader("Chat with the AI:")
chat_prompt = st.text_input("Ask me anything about finance, market trends, or anything else!")
if chat_prompt:
    with st.spinner("Getting response from AI..."):
        chat_response = general_ai_chat(chat_prompt)
        st.write(chat_response)

st.markdown("---") # Add a separator

# Input section using columns
col1, col2 = st.columns(2)

with col1:
    popular_stocks = ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'AMZN', 'NVDA', 'META']
    ticker_input = st.selectbox("Select a stock ticker:", popular_stocks).upper()

with col2:
    time_period = st.selectbox(
        "Select chart time period:",
        ('1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'),
        index=2
    )

# State variable to control visibility of analysis results
if 'analysis_done' not in st.session_state:
    st.session_state.analysis_done = False
if 'stock_info' not in st.session_state:
    st.session_state.stock_info = None

# Analyze button
if st.button("Analyze Stock"):
    if not ticker_input:
        st.warning("Please enter or select a stock ticker.")
    else:
        with st.spinner(f"Fetching data and generating narrative for {ticker_input}..."):
            price_change, hist, recommendations, news_text, info = fetch_stock_data(ticker_input, time_period)

            if hist is not None:
                bullish, bearish = mock_sentiment_score(ticker_input)
                narrative = generate_narrative(ticker_input, price_change, bullish, bearish, news_text, recommendations)

                # Store results in session state
                st.session_state.analysis_done = True
                st.session_state.stock_info = {
                    'ticker': ticker_input,
                    'price_change': price_change,
                    'bullish': bullish,
                    'bearish': bearish,
                    'narrative': narrative,
                    'hist': hist,
                    'recommendations': recommendations,
                    'news_text': news_text,
                    'info': info
                }
                # Rerun to display results
                st.rerun()

# Display Results if analysis is done
if st.session_state.analysis_done and st.session_state.stock_info:
    info_data = st.session_state.stock_info
    st.subheader(f"Analysis for {info_data['ticker']}")

    # Use columns for key metrics
    metric_col1, metric_col2 = st.columns(2)
    with metric_col1:
        st.metric(label="Price Change (Last Day)", value=f"{info_data['price_change']}%")
    with metric_col2:
        st.metric(label="Sentiment", value=f"📈 {info_data['bullish']}% Bullish, 📉 {info_data['bearish']}% Bearish")

    # Buttons for additional info (always visible after analysis)
    st.subheader("Additional Stock Information:")
    info_buttons_col1, info_buttons_col2, info_buttons_col3 = st.columns(3)

    with info_buttons_col1:
        if st.button("Show Market Cap"):
             market_cap = info_data['info'].get('marketCap')
             if market_cap is not None:
                 st.write(f"Market Cap: ${market_cap:,}")
             else:
                 st.write("Market Cap: N/A")

    with info_buttons_col2:
        if st.button("Show Dividend Yield"):
            dividend_yield = info_data['info'].get('dividendYield')
            if dividend_yield is not None:
                st.write(f"Dividend Yield: {dividend_yield:.2%}")
            else:
                st.write("Dividend Yield: N/A")

    with info_buttons_col3:
         if st.button("Show P/E Ratio"):
             pe_ratio = info_data['info'].get('trailingPE')
             if pe_ratio is not None:
                 st.write(f"P/E Ratio: {pe_ratio:.2f}")
             else:
                 st.write("P/E Ratio: N/A")


    st.subheader("Market Narrative:")
    st.write(info_data['narrative'])

    st.subheader(f"{info_data['ticker']} Price Chart ({time_period})")
    st.line_chart(info_data['hist']['Close'])

    st.subheader("Analyst Recommendations (Latest):")
    recommendations = info_data['recommendations']
    if recommendations is not None and not recommendations.empty:
        latest_rec = recommendations.iloc[0]
        st.write(f"Strong Buy: {latest_rec.get('strongBuy', 'N/A')}, Buy: {latest_rec.get('buy', 'N/A')}, Hold: {latest_rec.get('hold', 'N/A')}, Sell: {latest_rec.get('sell', 'N/A')}, Strong Sell: {latest_rec.get('strongSell', 'N/A')}")
    else:
        st.write("No recent analyst recommendations available.")

    st.subheader("Recent News Headlines:")
    news_text = info_data['news_text']
    if news_text:
        for headline in news_text.split(". "):
             st.write(f"- {headline}")
    else:
        st.write("No recent news available.")


st.info("This is an MVP. Features like real insider trading data, live sentiment analysis, and personalized narratives are under development.")

Overwriting app.py
